<a href="https://colab.research.google.com/github/salvapineda/notebooks/blob/main/PyomoNeos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Accessing computational time in PYOMO with NEOS

## Requirements

In [ ]:
!pip install -q pyomo
!apt-get install -y -qq coinor-cbc
import os
import random
import pyomo.environ as pe
os.environ['NEOS_EMAIL'] = 'xxx@gmail.com'

     |████████████████████████████████| 9.1 MB 5.6 MB/s 
     |████████████████████████████████| 49 kB 4.4 MB/s 
Selecting previously unselected package coinor-libcoinutils3v5.
(Reading database ... 155219 files and directories currently installed.)
Preparing to unpack .../0-coinor-libcoinutils3v5_2.10.14+repack1-1_amd64.deb ...
Unpacking coinor-libcoinutils3v5 (2.10.14+repack1-1) ...
Selecting previously unselected package coinor-libosi1v5.
Preparing to unpack .../1-coinor-libosi1v5_0.107.9+repack1-1_amd64.deb ...
Unpacking coinor-libosi1v5 (0.107.9+repack1-1) ...
Selecting previously unselected package coinor-libclp1.
Preparing to unpack .../2-coinor-libclp1_1.16.11+repack1-1_amd64.deb ...
Unpacking coinor-libclp1 (1.16.11+repack1-1) ...
Selecting previously unselected package coinor-libcgl1.
Preparing to unpack .../3-coinor-libcgl1_0.59.10+repack1-1_amd64.deb ...
Unpacking coinor-libcgl1 (0.59.10+repack1-1) ...
Selecting previously unselected package coinor-libcbc3.
Preparing to unp

## Create the optimization model in pyomo

In [ ]:
# Dimensions
nvar = 10
ncon = 5
# Random vector and matrixes
c = [round(random.uniform(0,1),2) for i in range(nvar)]
A = [[round(random.uniform(-1,1),2) for i in range(nvar)] for j in range(ncon)]
b = [round(random.uniform(-10,10),2) for j in range(ncon)]
# Model
m = pe.ConcreteModel()
# Sets
m.i = pe.Set(initialize=range(nvar),ordered=True)
m.j = pe.Set(initialize=range(ncon),ordered=True)
# Variables
m.x = pe.Var(m.i,within=pe.NonNegativeIntegers)
# Objective function
def obj_rule(m):
  return sum(c[i]*m.x[i] for i in m.i)
m.obj = pe.Objective(rule=obj_rule)
# Constraints
def con_rule(m,j):
  return sum(A[j][i]*m.x[i] for i in m.i) <= b[j]
m.con = pe.Constraint(m.j,rule=con_rule)

## Solve the model using CBC without NEOS

In [ ]:
pe.SolverFactory('cbc', executable='/usr/bin/cbc').solve(m).write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 7.71
  Upper bound: 7.71
  Number of objectives: 1
  Number of constraints: 5
  Number of variables: 10
  Number of binary variables: 0
  Number of integer variables: 10
  Number of nonzeros: 10
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.07
  Wallclock time: 0.08
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound: 
 

## Solve the model using CBC and NEOS

In [ ]:
pe.SolverManagerFactory('neos').solve(m,opt='cbc').write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 5
  Number of variables: 10
  Sense: unknown
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Message: CBC 2.10.5 optimal, objective 7.71; 32 nodes, 513 iterations, 0.047165 seconds
  Termination condition: optimal
  Id: 0
# ----------------------------------------------------------
#   Solution Information
# ----------------------------------------------------------
Solution: 
- number of solutions: 0
  number of solutions displayed: 0
